# **Reti Neurali** 

## **Esercizio 1: Download e pre-processamento dei dati.**

Scaricare e pre-processare i dati per il successivo addestramento del modello. 

Il dataset che utilizzeremo sarà CIFAR10, scaricabile dalla libreria `tensorflow.keras.datasets`

In [5]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10

# Download dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

y_train = y_train.ravel()
y_test = y_test.ravel()

# Stampare le shape dei dati
print("Shape di x_train:", x_train.shape)
print("Shape di y_train:", y_train.shape)
print("Shape di x_test:", x_test.shape)
print("Shape di y_test:", y_test.shape)

KeyboardInterrupt: 

In [ ]:
from sklearn.preprocessing import StandardScaler

# Pre-processamento dei dati
x_train = x_train.reshape((x_train.shape[0], -1))
x_test = x_test.reshape((x_test.shape[0], -1))

scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

## **Esercizio 2: Creare modello MLP**

Per creare il modello MLP utilizziamo l' oggetto `MLPClassifier` dal modulo `sklearn.neural_networks`. Questo è un oggetto molto complesso che prevede la possibilità di specificare tanti parametri, permettendoci una personalizzazione molto dettagliata. Vediamo di seguito gli argomenti principali:

- `hidden_layer_sizes`: rappresenta la struttura dell' MLP, sotto forma di una tupla. La tupla deve essere composta da numeri interi, ogni numero indica il numero di neuroni presenti nel rispettivo layer.

Esempio: 

`hidden_layer_sizes` = `(100)`

creerà un solo layer con 100 neuroni

`hidden_layer_sizes` = `(100, 50)`

creerà due layer, il primo con 100 neuroni, il secondo invece con 50.

- `max_iter`: massimo numero di iterazioni per raggiungere la convergenza. 

- `activation`: indica quale funzione di attivazione utilizzare, valori possibili sono `'relu'`, `'logistic'`, `'tanh'` and `'identity'`.

- `solver`: indica quale algoritmo di ottimizzazione utilizzare, valori possibili sono `'adam'`, `'sgd'` and `'lbfgs'`.

- `learning_rate_init`: valore iniziale del learning rate.

- `verbose`: valore booleano che, se impostato su `True`, stampa l' output di ogni iterazione di training. Molto utile per monitorare il training.

- `random_state`: fissa il seed della randomizzazione.


Per iniziare creiamo un MLP molto basilare, alleniamolo e testiamone le performance. Come parametri utilizzeremo:

- `hidden_layer_sizes` = `(100)`

- `max_iter` = `20`

- `random_state` = `42`

In [6]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

# Creare MLP 
mlp = MLPClassifier(hidden_layer_sizes=(100,), max_iter=20, random_state=42)

# Allenare MLP
mlp.fit(x_train, y_train)

# Valutare MLP
y_pred = mlp.predict(x_test)

# Stampare l' accuratezza
accuracy = accuracy_score(y_test, y_pred)
print("Accuratezza:", accuracy)

Accuratezza: 0.5057


/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  warnings.warn(


## **Esercizio 2.1: Aumentiamo i parametri del nostro modello**

Proviamo adesso ad aumentare i dettagli del nostro modello, modificando o aggiungendo i parametri sopra specificati. 

In [7]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

# Creare MLP con più strati e altre specifiche
mlp_deep = MLPClassifier(hidden_layer_sizes=(256, 128, 64), activation='relu', solver='adam', max_iter=50, random_state=42)
mlp_deep.fit(x_train, y_train)

/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (50) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPClassifier(hidden_layer_sizes=(256, 128, 64), max_iter=50, random_state=42)

## **Esercizio 3: Implementare manualmente l' algoritmo di early stopping.**

L' algoritmo di early stopping ci permette di terminare anticipatamente l' allenamento di un modello nel caso in cui questo raggiunga la convergenza. Supponiamo infatti che il nostro modello raggiunga un certo livello di accuratezza e che non riesca a migliorare oltre quel livello. Questo significa che il modello, da quel momento in poi, non sta più apprendendo nuove informazioni, per cui le successive iterazioni sono superflue, ed inoltre rischiano di essere dannose, spingendo il modello verso l' overfitting. 

L' early stopping verifica ad ogni iterazione che l' accuratezza del modello sia incrementata di una certa tolleranza. Se questa tolleranza non viene superata per un certo numero di epoche, allora possiamo decidere di stoppare l' allenamento in quanto il modello ha raggiunto la convergenza.

**N.B: per applicare early stopping è necessario specificare i seguenti parametri dell' MLP:**

- `warm_start`=`True` in modo che il training proceda dallo stato attuale del modello e non dall' inizializzazione.

- `max_iter`=`1` in modo che il modello venga allenato per una sola epoca. Per l' early stopping infatti dovremo gestire manualmente il numero di iterazioni.

In [9]:
import numpy as np
from sklearn.neural_network import MLPClassifier 
from sklearn.metrics import accuracy_score 

n_total_epochs = 100  
patience = 10         
tolerance = 1e-4      

best_test_accuracy = 0.0
epochs_without_improvement = 0

classes = np.unique(y_train)

mlp_partial = MLPClassifier(hidden_layer_sizes=(100,), max_iter=1, warm_start=True, random_state=42)

for epoch in range(n_total_epochs):
    if epoch == 0:
        mlp_partial.partial_fit(x_train, y_train, classes=classes)
    else:
        mlp_partial.partial_fit(x_train, y_train)

    y_test_pred = mlp_partial.predict(x_test)
    test_accuracy = accuracy_score(y_test, y_test_pred)

    if test_accuracy > best_test_accuracy + tolerance:
        best_test_accuracy = test_accuracy
        epochs_without_improvement = 0
    else:
        epochs_without_improvement += 1

    print(f"Epoch {epoch+1}, Accuracy: {test_accuracy:.4f}")

    if epochs_without_improvement >= patience:
        print("Early stopping attivato.")
        break

Epoch 1, Accuracy: 0.4325
Epoch 2, Accuracy: 0.4713
Epoch 3, Accuracy: 0.4756
Epoch 4, Accuracy: 0.4776
Epoch 5, Accuracy: 0.4812
Epoch 6, Accuracy: 0.4835
Epoch 7, Accuracy: 0.4845
Epoch 8, Accuracy: 0.4899
Epoch 9, Accuracy: 0.4890
Epoch 10, Accuracy: 0.4927
Epoch 11, Accuracy: 0.4916
Epoch 12, Accuracy: 0.4919
Epoch 13, Accuracy: 0.4927
Epoch 14, Accuracy: 0.4921
Epoch 15, Accuracy: 0.4900
Epoch 16, Accuracy: 0.4895
Epoch 17, Accuracy: 0.4888
Epoch 18, Accuracy: 0.4875
Epoch 19, Accuracy: 0.4868
Epoch 20, Accuracy: 0.4816
Early stopping attivato.
